# 03 - Assemble the RAG Pipeline

In [ ]:
from collections import defaultdict
import pandas as pd
import numpy as np
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.schema import Document

from archive import prompt_template
from utils.vector_store import VectorDB
from utils.io_utils import get_absolute_path
import torch

On initialise le retriever, pour comprendre comment il fonctionne, on peut regarder le notebook `notebooks/scripts/02.ipynb` qui permet de créer un vectordb à partir d'un dataframe.


In [ ]:
# init variables for retriever just for example (check available in config.yml for more details)
backend = "lancedb"
embedding_model_name = "Lajavaness/sentence-camembert-large"
k= 5
path = get_absolute_path("notebooks/scripts/data/vectordb/camembert-large_lancedb")
db = VectorDB(
    backend=backend,
    embedding_model=embedding_model_name,
    path=path
)

On teste le retriever

In [ ]:
# Requête
query = ("Pourquoi trouve-t-on des images dans les marges des manuscrits au Moyen-Âge ?")
results = db.query(query, k=10)  # results = List[Tuple[Document, float]]

# Groupement par file_id
def group_by_thesis(results_with_score):
    grouped = defaultdict(list)
    for doc, score in results_with_score:
        file_id = doc.metadata.get("file_id", "unknown")
        grouped[file_id].append((doc, score))
    return grouped

# Affichage structuré
def readeable_output_display(grouped_results):
    for file_id, docs in grouped_results.items():
        # Trier par score descendant
        docs = sorted(docs, key=lambda tup: tup[1], reverse=True)

        titre = docs[0][0].metadata.get("position_name", "Sans titre")
        auteur = docs[0][0].metadata.get("author", "Inconnu")
        date = docs[0][0].metadata.get("year", "Inconnu")

        print(f"\n📄 {auteur}, {titre}, promotion {date}")
        print(f"🧩 Chunks trouvés : {len(docs)}\n")

        for i, (doc, score) in enumerate(docs):
            extrait = doc.metadata.get("raw_chunk", doc.page_content).strip().replace("\n", " ")
            section = doc.metadata.get("section", "Inconnu")
            print(f"  ▪️ Extrait {i + 1} de la section '{section}' (score={score:.4f}) : […]{extrait}[…]")
        print("-" * 80)

# Exécution
readeable_output_display(group_by_thesis(results))


## Reader

Le reader est le composant du RAG qui va prendre les résultats du retriever et les transformer (interpolation) en un prompt pré-structuré et le passer un LLM qui en retour génèrera une réponse structurée ou non.

Les paramètres à prendre compte pour le reader sont :
- Le modèle LLM qui va être utilisé pour générer la réponse ;
- Le prompt qui va être utilisé pour générer la réponse ;

Pour initialiser le Reader on passe par l'interface `ChatOpenAI` de LangChain qui permet d'utiliser les modèles LLM d'OpenAI (ou compatibles avec la specication d'API OpenAI, comme Mistral, Llama, etc.).
Le LLM est servi par LM Studio qui est un serveur compatible avec l'API OpenAI et qui permet d'utiliser des modèles LLM locaux facilement (il existe d'autres solutions comme Ollama ou Vllm par exemple)

Pour initialiser le LLM on doit spécifier (voir la doc https://python.langchain.com/docs/integrations/chat/openai/) :
- Le nombre maximum de tokens à générer par le LLM (`max_tokens`) ;
- Le temperature du LLM (pour la créativité de la réponse). Une temperature de 0.0 rendra le LLM très déterministe, tandis qu'une température de 1.0 le rendra plus créatif et aléatoire (`temperature`).
- L'url de l'API OpenAI (qui est en fait l'url du serveur LM Studio) (`openai_api_base`) ;
- La reponse sera en streaming (`streaming=True`) pour afficher la réponse au fur et à mesure de sa génération.
- max_retries : le nombre de tentatives de génération en cas d'erreur (par exemple, si le LLM ne répond pas dans un délai raisonnable) (`max_retries`).
- timeout : le délai maximum d'attente pour une réponse du LLM (`timeout`).


In [ ]:
llm = ChatOpenAI(
    model_name="mistral-nemo-instruct-2407",
    openai_api_base="http://localhost:1234/v1",
    openai_api_key="lm-studio",
    temperature=0.0,
    streaming=True,
    max_tokens=4096,
)

Create a prompt template for the reader

Il n'est exetensible 3 paramètres :
- output_buffer : le buffer de sortie du reader
- max_tokens : le nombre maximum de tokens à générer
- embeding_max_tokens : le nombre maximum d

In [ ]:
from typing import List, Tuple
from collections import defaultdict
from jinja2 import Template
import tiktoken
from nltk.tokenize import sent_tokenize
from langchain_core.documents import Document
import lmstudio as lms


# 1. Fonction pour compter les tokens
def get_token_count(text: str) -> int:
    model = lms.llm(
    )
    return len(model.tokenize(text))

# 2. Troncature douce : on enlève des phrases à la fin
def truncate_by_sentence(text: str, max_tokens: int) -> str:
    sentences = sent_tokenize(text)
    result = []
    for sent in sentences:
        result.append(sent)
        joined = " ".join(result)
        if get_token_count(joined) > max_tokens:
            result.pop()
            break
    return " ".join(result).strip() + " […]"

# 3. Génération du contexte structuré, avec section annexe
def build_context_prompt(
    results: List[Tuple[Document, float]],
    question: str,
    template_path: str,
    max_total_tokens: int = 4096,
    output_buffer: int = 512,
    max_chunk_tokens: int = 500
) -> str:
    prompt_template = Template(open(template_path, encoding="utf-8").read())

    grouped = defaultdict(list)
    for doc, score in results:
        grouped[doc.metadata.get("file_id", "inconnu")].append((doc, score))

    # test if all score is under < 0.5:
    if all(score < 0.5 for _, score in results):
        return "no documents found"

    sorted_groups = sorted(grouped.items(), key=lambda item: max(s for _, s in item[1]), reverse=True)

    included_chunks = []
    fallback_chunks = []
    token_budget = max_total_tokens - output_buffer

    header_base = prompt_template.render(context="PLACEHOLDER", question="PLACEHOLDER", annex="PLACEHOLDER")
    header_tokens = get_token_count(header_base.replace("{{context}}", "").replace("{{question}}", ""))

    total_tokens = header_tokens

    for file_id, chunks in sorted_groups:
        chunks.sort(key=lambda x: x[1], reverse=True)
        meta = chunks[0][0].metadata
        header = f"* Position de thèse : {meta.get('author','?')}, {meta.get('position_name','?')}, promotion {meta.get('year','?')}\n"
        section_lines = [header]

        for i, (doc, _) in enumerate(chunks):
            section = doc.metadata.get("section", "")
            extrait = doc.metadata.get("raw_chunk", doc.page_content).strip().replace("\n", " ")
            #extrait = truncate_by_sentence(extrait, max_chunk_tokens)

            line = f"Extrait {i+1}"
            if section:
                line += f" - section « {section} »"
            line += f" : {extrait}"

            chunk_tokens = get_token_count(line)
            if total_tokens + chunk_tokens > token_budget:
                fallback_chunks.append((meta, i+1, section))
                continue

            section_lines.append(line)
            total_tokens += chunk_tokens

        if len(section_lines) > 1:
            included_chunks.append("\n".join(section_lines) + "\n")

    context = "\n".join(included_chunks)

    # Thèses éloignées
    if fallback_chunks:
        annex_by_thesis = defaultdict(list)
        for meta, i, section in fallback_chunks:
            file_id = meta.get("file_id", "inconnu")
            annex_by_thesis[file_id].append((meta, i, section))

        annex_lines = []
        for file_id, chunk_infos in annex_by_thesis.items():
            meta = chunk_infos[0][0]
            title = meta.get("position_name", "?")
            author = meta.get("author", "?")
            promo = meta.get("year", "?")
            header = f"* {author}, {title}, promotion {promo} :"
            annex_lines.append(header)

            for _, i, section in chunk_infos:
                line = f"\t- "
                if section:
                    line += f"section « {section} »"
                annex_lines.append(line)

        annex = "\n".join(annex_lines)
    else:
        annex = None

    return prompt_template.render(context=context, question=question, annex=annex)

final_prompt = build_context_prompt(
    results=results,
    question=query,
    template_path="../prompt_templates/v3.jinja",
    max_total_tokens=4096,
    output_buffer=512,
    max_chunk_tokens=500
)

print(final_prompt)

On passe le prompt au LLM pour obtenir la réponse

In [ ]:
import asyncio
import logging
import re
from IPython.display import display, Markdown
from spellchecker import SpellChecker
import language_tool_python
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
tool = language_tool_python.LanguageTool('fr')

def mask_capitalized_words(text):
    masks = {}
    def replacer(match):
        key = f"__MASK{len(masks)}__"
        masks[key] = match.group(0)
        return key

    # Masque tous les mots qui commencent par une majuscule (hors début de phrase si souhaité)
    pattern = r'\b[A-ZÉÈÀÂÊÎÔÛÇ][a-zéèêàçîôûäëïöü]+\b'
    masked_text = re.sub(pattern, replacer, text)
    return masked_text, masks

def unmask_text(text, masks):
    for key, original in masks.items():
        text = text.replace(key, original)
    return text



CONCLUSION = (
    "Pour rappel, cette réponse est générée automatiquement à partir d'un modèle de langue elle peut contenir des approximations, des surcorrections, des erreurs factuelles "
    "ou des interprétations partielles. Cette réponse ne prétend pas se subsituer à la critique des sources. Dans ce cadre, il est vivement recommandé de vérifier les sources mentionnées "
    "dans cette réponse et de consulter d'autres positions de thèses pour approfondir votre question."
)

def finalize_response(text: str, conclusion: str = CONCLUSION) -> str:
    lines = text.strip().split('\n')
    final_lines = []

    for line in lines:
        sentences = re.split(r'(?<=[.!?])\s+', line)
        if not sentences:
            continue
        if not re.search(r'[.!?]["”»”]?\s*$', sentences[-1]):
            sentences = sentences[:-1]
        if sentences:
            final_lines.append(" ".join(sentences))

    cleaned_text = "\n".join(final_lines).strip()
    if not cleaned_text:
        return conclusion

    # Correction avec masquage
    masked_text, masks = mask_capitalized_words(cleaned_text)
    matches = tool.check(masked_text)
    corrected = language_tool_python.utils.correct(masked_text, matches)
    corrected_text = unmask_text(corrected, masks)

    return corrected_text + "\n\n" + conclusion

# Fonction de génération avec stream
async def stream_and_print(llm, prompt, max_tokens=100000, timeout=100000):
    full_response = ""
    output_display = display(Markdown("🟡 Génération en cours..."), display_id=True)

    async def _run_stream():
        nonlocal full_response
        chunks = []
        async for chunk in llm.astream(prompt):
            text = getattr(chunk, "content", str(chunk))
            full_response += text
            chunks.append(text)
            output_display.update(Markdown("".join(chunks).replace("\n", "\n\n")))
            if len(full_response.split()) > max_tokens:
                output_display.update(Markdown("⛔️ **Réponse tronquée : nombre de tokens dépassé.**"))
                break

    try:
        await asyncio.wait_for(_run_stream(), timeout=timeout)
    except asyncio.TimeoutError:
        logging.warning("⏳ Timeout — relance en stream...")
        output_display.update(Markdown("🔁 **Reprise après timeout...**"))
        full_response = ""
        try:
            await asyncio.wait_for(_run_stream(), timeout=timeout)
        except Exception as e:
            logging.error(f"❌ Échec du deuxième stream : {e}")
            output_display.update(Markdown("❌ **Impossible de générer une réponse actuellement.**"))
            return "Erreur"
    except Exception as e:
        logging.warning(f"💥 Erreur pendant le stream : {e}")
        output_display.update(Markdown("🔁 **Nouvelle tentative de génération...**"))
        full_response = ""
        try:
            await asyncio.wait_for(_run_stream(), timeout=timeout)
        except Exception as e:
            logging.error(f"❌ Échec de la seconde tentative : {e}")
            output_display.update(Markdown("❌ **Impossible de générer une réponse actuellement.**"))
            return "Erreur"

    # Nettoyage et finalisation
    cleaned_response = finalize_response(full_response)
    output_display.update(Markdown(cleaned_response.replace("\n", "\n\n")))
    return cleaned_response

# Utilisation
if final_prompt == "no documents found":
    response = "Je n'ai trouvé aucun document pertinent pour répondre à votre question. Veuillez reformuler votre question ou bien consulter les positions de thèses disponibles."
    # stream this
    output_display = display(Markdown(response), display_id=True)
else:
    response = await stream_and_print(llm, final_prompt)

with open("response.txt", "w", encoding="utf-8") as f:
    f.write(response)


Il existe deux optimisations possibles :

- hybrid search : algorithme classique de recherche d'information (par exemple, BM25) combiné avec un retriever vectoriel (comme FAISS ou Qdrant) pour améliorer la pertinence des résultats.
Les algorthmes classiques sont souvent très bon pour détecter la présence / absence de mots-clés dans les documents tandis que les algorithmes vectoriels sont meilleurs pour détecter la similarité sémantique entre les documents et la question posée.
On combine donc les deux approches pour obtenir des résultats plus pertinents. Ceci est possible via `EnsembleRetriever` de LangChain qui permet de combiner plusieurs retrievers en un seul en se basant sur un algorithme de recherche hybride qui réalise une combinaison pondérée des résultats de chaque retriever.

- reranking : cette métode permet de réordonner les résultats obtenus par le retriever en fonction de la pertinence par rapport à la question posée en utilisant un modèle de langage avant le renvoi final des résultats par le retriever. L'idée étant que le modèle de langage peut mieux comprendre le contexte et la pertinence des documents par rapport à la question posée, et ainsi réorganiser les résultats pour renvoyer les plus pertinents en premier.

Comme la classe VectorDB nous avons créer une abstration `RAGPipeline` qui inclut le Retrievier via `VectorDB`, la logique vu précédément ainsi que la possibilité d'utilisé l'hybrid search et le reranking.

Dans le notebook sur l'évaluation nous comparons les performances de ces deux approches que l'ont peut combiner ou non.

In [ ]:
from utils.rag_pipeline import RAGPipeline

In [ ]:
pipeline = RAGPipeline(
    vectordb_path="./scripts/data/vectordb/camembert-base_faiss",
    template_path="../prompt_templates/v3.jinja",
    backend="faiss",
    embedding_model=None,
    hybrid=True,
    bm25_path="../data/vectordb/bm25/bm25.encpos.tok.512_51.pkl",
    rerank=False,
    use_streaming=True
)

In [ ]:
results = await pipeline.generate("Quelle est la place de l'héraldique dans les thèses du XIXe siècle ?")
for doc, score in pipeline.relevant_docs:
    print(f"[{score:.2f}] {doc.metadata.get('author', '?')} - {doc.metadata.get('section', '?')}")
    print(doc.page_content[:300], "...\n")

In [1]:
import time
from utils.rag_pipeline import RAGPipeline  # adapte si nécessaire

# Paramètres communs
template_path = "../prompt_templates/v3.jinja"
query = "Quelle est la place de l'héraldique dans les thèses du XIXe siècle ?"
bm25_path = "../data/vectordb/bm25/bm25.encpos.tok.512_51.pkl"

configs = {
    "BM25 uniquement": dict(
        backend="faiss",
        vectordb_path=None,
        embedding_model=None,
        bm25_path=bm25_path,
        hybrid=False,
        rerank=False
    ),
    "Vectorstore FAISS": dict(
        backend="faiss",
        vectordb_path="./scripts/data/vectordb/camembert-base_faiss",
        embedding_model="Lajavaness/sentence-camembert-base",
        bm25_path=None,
        hybrid=False,
        rerank=False
    ),
    "Vectorstore LanceDB": dict(
        backend="lancedb",
        vectordb_path="./scripts/data/vectordb/camembert-base_lancedb",
        embedding_model="Lajavaness/sentence-camembert-base",
        bm25_path=None,
        hybrid=False,
        rerank=False
    ),
    "Hybrid FAISS": dict(
        backend="faiss",
        vectordb_path="./scripts/data/vectordb/camembert-base_faiss",
        embedding_model="Lajavaness/sentence-camembert-base",
        bm25_path=bm25_path,
        hybrid=True,
        rerank=False
    ),
    "Hybrid LanceDB": dict(
        backend="lancedb",
        vectordb_path="./scripts/data/vectordb/camembert-base_lancedb",
        embedding_model="Lajavaness/sentence-camembert-base",
        bm25_path=bm25_path,
        hybrid=True,
        rerank=False
    ),
    "Rerank FAISS": dict(
        backend="faiss",
        vectordb_path="./scripts/data/vectordb/camembert-base_faiss",
        embedding_model="Lajavaness/sentence-camembert-base",
        bm25_path=None,
        hybrid=False,
        rerank=True
    ),
    "Rerank LanceDB": dict(
        backend="lancedb",
        vectordb_path="./scripts/data/vectordb/camembert-base_lancedb",
        embedding_model="Lajavaness/sentence-camembert-base",
        bm25_path=None,
        hybrid=False,
        rerank=True
    ),
    "Hybrid + Rerank FAISS": dict(
        backend="faiss",
        vectordb_path="./scripts/data/vectordb/camembert-base_faiss",
        embedding_model="Lajavaness/sentence-camembert-base",
        bm25_path=bm25_path,
        hybrid=True,
        rerank=True
    ),
    "Hybrid + Rerank LanceDB": dict(
        backend="lancedb",
        vectordb_path="./scripts/data/vectordb/camembert-base_lancedb",
        embedding_model="Lajavaness/sentence-camembert-base",
        bm25_path=bm25_path,
        hybrid=True,
        rerank=True
    )
}

import pandas as pd
results = []

for label, kwargs in configs.items():
    print(f"\n=== {label} ===")
    pipeline = RAGPipeline(
        template_path=template_path,
        use_streaming=False,
        use_notebook=False,
        k=5,
        **kwargs
    )
    start = time.time()
    response = await pipeline.generate(query)
    duration = time.time() - start
    print(f"\n⏱️ Temps écoulé : {duration:.2f} secondes")
    print(f"📄 Documents trouvés : {len(pipeline.relevant_docs)}")

    print("\n🔍 Top 3 documents :")
    top = pipeline.relevant_docs[:3]
    for doc, score in top:
        print(f" - [{score:.2f}] {doc.metadata.get('author', '?')} | {doc.metadata.get('section', '?')}")
        print(f"   {doc.page_content[:200]}...\n")

    results.append({
        "Configuration": label,
        "Durée (s)": round(duration, 2),
        "Nb docs": len(pipeline.relevant_docs)
    })

df = pd.DataFrame(results)
print("\n📊 Récapitulatif :")
print(df.to_markdown(index=False))



=== BM25 uniquement ===

✅ Réponse générée.
L'héraldique, qui est l'étude des blasons et des armoiries, semble avoir joué un rôle relativement limité dans les positions de thèse étudiées. Les extraits fournis ne font pas directement référence à cette discipline.
Dans la position de thèse de Florence Köll sur le Palais-Royal, bien que l'on puisse trouver des références à l'héraldique dans le contexte du XVIIIe siècle, il n'y a pas de mention spécifique de cette discipline dans les événements révolutionnaires et impériaux qui ont suivi. L'étude se concentre plutôt sur la gestion du domaine, les travaux du palais princier, les locations, les problèmes d'hygiène et de police, ainsi que sur l'utilisation du Tribunat et de la Bourse.
Dans la position de thèse de Claire Haquet sur les « sages marchands et bourgeois de Rouen », l'accent est mis sur l'histoire urbaine en France à la fin du Moyen Âge, avec une étude d'ensemble sur Paris et Rouen. L'héraldique n'est pas mentionnée comme un éléme